In [1]:
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

%matplotlib inline

from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, train_test_split

import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype, is_numeric_dtype

import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold

from rfpimp import *

from sklearn.metrics import roc_auc_score

from sortedcontainers import SortedList
import copy
import collections
import numpy as np
from itertools import product, chain
import pandas
from sklearn.model_selection import KFold
import catboost as cb
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import scale
# from sklearn.cross_validation import *
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, ParameterGrid, train_test_split
from sklearn.metrics import fbeta_score, make_scorer,recall_score,precision_score
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
import gzip
import warnings
warnings.filterwarnings(category=DeprecationWarning,action='ignore')
import pandas as pd

from sklearn.model_selection import GridSearchCV

import sklearn.metrics as metrics
import matplotlib.pyplot as plt

from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import numpy as np

# xgboost

In [17]:
path = 'data/mckinsey/'

train = pd.read_csv(path + 'train.csv')

test = pd.read_csv(path + 'test.csv')

In [ ]:
sample = pd.read_csv(path + 'sample_submission_1.csv')

train.head(1)

test.head(1)

train.shape

train.isnull().sum()

train.smoking_status.unique()

train.work_type.unique()

train.Residence_type.unique()

train.groupby('stroke').size()

def train_cats(df):
    """Change any columns of strings in a panda's dataframe to a column of
    catagorical values. This applies the changes inplace.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values.

    Examples:
    ---------

    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category
    """
    for n, c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

train_cats(train)

def apply_cats(df, trn):
    """Changes any columns of strings in df into categorical variables using trn as
    a template for the category codes.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values. The category codes are determined by trn.

    trn: A pandas dataframe. When creating a category for df, it looks up the
        what the category's code were in trn and makes those the category codes
        for df.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category {a : 1, b : 2}

    >>> df2 = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['b', 'a', 'a']})
    >>> apply_cats(df2, df)

           col1 col2
        0     1    b
        1     2    a
        2     3    a

    now the type of col is category {a : 1, b : 2}
    """
    for n, c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name == 'category'):
            df[n] = pd.Categorical(
                c, categories=trn[n].cat.categories, ordered=True)

apply_cats(test, train)

train['bmi_is_na'] = 0

train.loc[train.bmi.isna(),'bmi_is_na'] = 1

test['bmi_is_na'] = 0

test.loc[test.bmi.isna(),'bmi_is_na'] = 1

train.bmi.fillna(train.bmi[train.bmi.notnull()].median(), inplace=True)

test.bmi.fillna(train.bmi[train.bmi.notnull()].median(), inplace=True)

train.dtypes

test.dtypes

cat_cols = [
    'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'
]

for c in cat_cols:
    train[c] = train[c].cat.codes
    test[c] = test[c].cat.codes

train.head()

test.head()

def create_submission(pred, path, fname):
    submission = pd.DataFrame({'id': test.id, 'stroke': pred})
    submission.to_csv(path + fname, index=False)

train.shape

test.shape

def modelfit(alg,
             labels,
             predictors,
             useTrainCV=True,
             cv_folds=5,
             early_stopping_rounds=50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(predictors, labels)
        cvresult = xgb.cv(
            xgb_param,
            xgtrain,
            num_boost_round=alg.get_params()['n_estimators'],
            nfold=cv_folds,
            metrics='auc',
            early_stopping_rounds=early_stopping_rounds,
            show_stdv=True,
            verbose_eval=True,
            seed=4,stratified=True)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(predictors, labels)

    #Predict training set:
    dtrain_predictions = alg.predict(predictors)
    dtrain_predprob = alg.predict_proba(predictors)[:, 1]

    #Print model report:
#     fbeta_score = fbeta_threshold(labels, dtrain_predprob)
#     auc_score = metrics.roc_auc_score(labels, dtrain_predprob)

#     print(f"fbeta: {fbeta_score}")
#     print(f"AUC: {auc_score}")

    return alg

xgb1 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    seed=27,
    silent=True)

x_train, y_train = train.drop('stroke', axis=1).values, train.stroke

m = modelfit(xgb1,y_train,x_train,cv_folds=3)

m

param_test1 = {
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2)
}
gsearch1 = GridSearchCV(
    estimator=xgb.XGBClassifier(
        learning_rate=0.1,
        n_estimators=37,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27),
    param_grid=param_test1,
    scoring='roc_auc',
    n_jobs=4,
    iid=False,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        train.drop('stroke', axis=1), train.stroke),verbose=3)

gsearch1.fit(x_train, y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

param_test1 = {
    'max_depth': [2,3,4],
    'min_child_weight': [4,5,6]
}
gsearch1 = GridSearchCV(
    estimator=xgb.XGBClassifier(
        learning_rate=0.1,
        n_estimators=37,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27),
    param_grid=param_test1,
    scoring='roc_auc',
    n_jobs=4,
    iid=False,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        train.drop('stroke', axis=1), train.stroke),verbose=3)

gsearch1.fit(x_train, y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

param_test1 = {
    'max_depth': [3],
    'min_child_weight': [22,24,26]
}
gsearch1 = GridSearchCV(
    estimator=xgb.XGBClassifier(
        learning_rate=0.1,
        n_estimators=37,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27),
    param_grid=param_test1,
    scoring='roc_auc',
    n_jobs=4,
    iid=False,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        train.drop('stroke', axis=1), train.stroke),verbose=3)

gsearch1.fit(x_train, y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(
    estimator=xgb.XGBClassifier(
        learning_rate=0.1,
        n_estimators=37,
        max_depth=3,
        min_child_weight=22,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27),
    param_grid=param_test3,
    scoring='roc_auc',
    n_jobs=4,
    iid=False,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        train.drop('stroke', axis=1), train.stroke),verbose=3)

gsearch3.fit(x_train, y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

xgb1 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=3,
    min_child_weight=22,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    seed=27,
    silent=True)

x_train, y_train = train.drop('stroke', axis=1).values, train.stroke

m = modelfit(xgb1,y_train,x_train,cv_folds=3)

m

param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(
    estimator=xgb.XGBClassifier(
        learning_rate=0.1,
        n_estimators=37,
        max_depth=3,
        min_child_weight=22,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27),
    param_grid=param_test4,
    scoring='roc_auc',
    n_jobs=4,
    iid=False,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        train.drop('stroke', axis=1), train.stroke),verbose=3)

gsearch4.fit(x_train, y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

param_test5 = {
 'subsample':[i/100.0 for i in [85,90,95]],
 'colsample_bytree':[i/100.0 for i in [75,80,85]]
}
gsearch5 = GridSearchCV(
    estimator=xgb.XGBClassifier(
        learning_rate=0.1,
        n_estimators=37,
        max_depth=3,
        min_child_weight=22,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27),
    param_grid=param_test5,
    scoring='roc_auc',
    n_jobs=4,
    iid=False,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        train.drop('stroke', axis=1), train.stroke),verbose=3)

gsearch5.fit(x_train, y_train)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

xgb1 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=3,
    min_child_weight=22,
    gamma=0,
    subsample=0.9,
    colsample_bytree=0.75,
    objective='binary:logistic',
    seed=27,
    silent=True)

x_train, y_train = train.drop('stroke', axis=1).values, train.stroke

m = modelfit(xgb1,y_train,x_train,cv_folds=3)

m

m.fit(x_train,y_train)

probs = m.predict_proba(test.values)[:,1]

create_submission(probs,path,'xgb8.csv')

m

for stacking

x_train, y_train = train.drop('stroke', axis=1).reset_index(drop= True), train.stroke

x_test = test

clf = m

trn = x_train.copy()

tst = x_test.copy()

x_train['xgb_soft'] = 0

kf = KFold(n_splits=5, shuffle=True, random_state=1)

for train_index, val_index in StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        x_train, y_train):
    x_trn, y_trn = trn.loc[train_index,:], y_train[train_index]
    x_val, y_val = trn.loc[val_index,:], y_train[val_index]

    clf.fit(x_trn, y_trn)
    
    probs = clf.predict_proba(x_val)[:, 1]
    
    
    x_train.loc[val_index,'xgb_soft'] = probs

    
clf.fit(trn,y_train)
probs = clf.predict_proba(tst)[:, 1]

x_test.loc[:,'xgb_soft'] = probs

x_train.to_feather('data/mckinsey/stack_trn_xgb')

x_test.to_feather('data/mckinsey/stack_tst_xgb')

# Random Forest

In [ ]:
path = 'data/mckinsey/'

train = pd.read_csv(path + 'train.csv')

test = pd.read_csv(path + 'test.csv')

sample = pd.read_csv(path + 'sample_submission_1.csv')

train.head(1)

test.head(1)

train.shape

train.isnull().sum()

train.smoking_status.unique()

train.work_type.unique()

train.Residence_type.unique()

train.groupby('stroke').size()

def train_cats(df):
    """Change any columns of strings in a panda's dataframe to a column of
    catagorical values. This applies the changes inplace.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values.

    Examples:
    ---------

    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category
    """
    for n, c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

train_cats(train)

def apply_cats(df, trn):
    """Changes any columns of strings in df into categorical variables using trn as
    a template for the category codes.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values. The category codes are determined by trn.

    trn: A pandas dataframe. When creating a category for df, it looks up the
        what the category's code were in trn and makes those the category codes
        for df.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category {a : 1, b : 2}

    >>> df2 = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['b', 'a', 'a']})
    >>> apply_cats(df2, df)

           col1 col2
        0     1    b
        1     2    a
        2     3    a

    now the type of col is category {a : 1, b : 2}
    """
    for n, c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name == 'category'):
            df[n] = pd.Categorical(
                c, categories=trn[n].cat.categories, ordered=True)

apply_cats(test, train)

train['bmi_is_na'] = 0

train.loc[train.bmi.isna(),'bmi_is_na'] = 1

test['bmi_is_na'] = 0

test.loc[test.bmi.isna(),'bmi_is_na'] = 1

train.bmi.fillna(train.bmi[train.bmi.notnull()].median(), inplace=True)

test.bmi.fillna(train.bmi[train.bmi.notnull()].median(), inplace=True)

train.dtypes

test.dtypes

cat_cols = [
    'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'
]

for c in cat_cols:
    train[c] = train[c].cat.codes
    test[c] = test[c].cat.codes

train.head()

test.head()

def create_submission(pred, path, fname):
    submission = pd.DataFrame({'id': test.id, 'stroke': pred})
    submission.to_csv(path + fname, index=False)

train.shape

test.shape

### without class weight balanced

all features considered

model = RandomForestClassifier(
    random_state=1,
    n_jobs=-1,
    n_estimators=50,
    oob_score=False,
    min_samples_leaf=3)

grid = {
    'min_samples_leaf': [200,230,250],
    'max_features': ['auto']
}

clf = GridSearchCV(
    model,
    grid,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        train.drop('stroke', axis=1), train.stroke),
    scoring='roc_auc',
    verbose=5)

clf.fit(train.drop('stroke', axis=1), train.stroke)

clf.best_score_

clf.best_params_

rf = RandomForestClassifier(
    n_jobs=-1,
    n_estimators=200,
    min_samples_leaf=200,
    max_features='auto',
    oob_score=True)

rf.fit(train.drop('stroke', axis=1), train.stroke)

probs = rf.predict_proba(test)[:,1]

create_submission(probs,path,'rfv5.csv')

feature selection

cols_to_keep = ['heart_disease', 'id', 'bmi', 'avg_glucose_level', 'age','hypertension']

x_train,y_train = train.loc[:,cols_to_keep], train.stroke

model = RandomForestClassifier(
    random_state=1,
    n_jobs=-1,
    n_estimators=50,
    oob_score=False,
    min_samples_leaf=3)

grid = {
    'min_samples_leaf': [200,230,250],
    'max_features': ['auto','log2',None,0.5]
}

clf = GridSearchCV(
    model,
    grid,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        x_train, y_train),
    scoring='roc_auc',
    verbose=5)

clf.fit(x_train, y_train)

clf.best_score_

clf.best_params_

cv score is not improving

without stratification

model = RandomForestClassifier(
    random_state=1,
    n_jobs=-1,
    n_estimators=50,
    oob_score=False,
    min_samples_leaf=3)

grid = {
    'min_samples_leaf': [100,150,200,230,250],
    'max_features': ['auto']
}

clf = GridSearchCV(
    model,
    grid,
    cv=3,
    scoring='roc_auc',
    verbose=5)

clf.fit(train.drop('stroke', axis=1), train.stroke)

clf.best_score_

clf.best_params_

### Ensembling

xgb = pd.read_csv('data/mckinsey/xgb_ens.csv')

rf = pd.read_csv('data/mckinsey/rf_ens.csv')

lgb = pd.read_csv('data/mckinsey/lgb_ens.csv')

cb = pd.read_csv('data/mckinsey/cat_ens.csv')

plt.hist(xgb.stroke)

plt.hist(rf.stroke)

plt.hist(lgb.stroke)

plt.hist(cb.stroke)

pred = (rf.stroke+xgb.stroke+cb.stroke+lgb.stroke)/4

create_submission(pred,path,'ensemble_4.csv')

for stacking

x_train, y_train = train.drop('stroke', axis=1).reset_index(drop= True), train.stroke

x_test = test

clf = RandomForestClassifier(
    n_jobs=-1,
    n_estimators=200,
    min_samples_leaf=200,
    max_features='auto',
    oob_score=True)

trn = x_train.copy()

tst = x_test.copy()

x_train['rf_soft'] = 0

kf = KFold(n_splits=5, shuffle=True, random_state=1)

for train_index, val_index in StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        x_train, y_train):
    x_trn, y_trn = trn.loc[train_index,:], y_train[train_index]
    x_val, y_val = trn.loc[val_index,:], y_train[val_index]

    clf.fit(x_trn, y_trn)
    
    probs = clf.predict_proba(x_val)[:, 1]
    
    
    x_train.loc[val_index,'rf_soft'] = probs

    
clf.fit(trn,y_train)
probs = clf.predict_proba(tst)[:, 1]

x_test.loc[:,'rf_soft'] = probs

x_train.to_feather('data/mckinsey/stack_trn_rf')

x_test.to_feather('data/mckinsey/stack_tst_rf')



# Light GBM

In [ ]:
path = 'data/mckinsey/'

train = pd.read_csv(path + 'train.csv')

test = pd.read_csv(path + 'test.csv')

sample = pd.read_csv(path + 'sample_submission_1.csv')

train.head(1)

test.head(1)

train.shape

train.isnull().sum()

train.smoking_status.unique()

train.work_type.unique()

train.Residence_type.unique()

train.groupby('stroke').size()

def train_cats(df):
    """Change any columns of strings in a panda's dataframe to a column of
    catagorical values. This applies the changes inplace.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values.

    Examples:
    ---------

    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category
    """
    for n, c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

train_cats(train)

def apply_cats(df, trn):
    """Changes any columns of strings in df into categorical variables using trn as
    a template for the category codes.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values. The category codes are determined by trn.

    trn: A pandas dataframe. When creating a category for df, it looks up the
        what the category's code were in trn and makes those the category codes
        for df.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category {a : 1, b : 2}

    >>> df2 = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['b', 'a', 'a']})
    >>> apply_cats(df2, df)

           col1 col2
        0     1    b
        1     2    a
        2     3    a

    now the type of col is category {a : 1, b : 2}
    """
    for n, c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name == 'category'):
            df[n] = pd.Categorical(
                c, categories=trn[n].cat.categories, ordered=True)

apply_cats(test, train)

train['bmi_is_na'] = 0

train.loc[train.bmi.isna(),'bmi_is_na'] = 1

test['bmi_is_na'] = 0

test.loc[test.bmi.isna(),'bmi_is_na'] = 1

train.bmi.fillna(train.bmi[train.bmi.notnull()].median(), inplace=True)

test.bmi.fillna(train.bmi[train.bmi.notnull()].median(), inplace=True)

train.dtypes

test.dtypes

cat_cols = [
    'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'
]

for c in cat_cols:
    train[c] = train[c].cat.codes
    test[c] = test[c].cat.codes

train.head()

test.head()

def create_submission(pred, path, fname):
    submission = pd.DataFrame({'id': test.id, 'stroke': pred})
    submission.to_csv(path + fname, index=False)

train.shape

test.shape

lg = lgb.LGBMClassifier(learning_rate=0.05, num_leaves=10, max_depth=15, subsample=0.8, n_estimators=100, )

lg.fit(train.drop('stroke', axis=1), train.stroke)

probs = lg.predict_proba(test)[:,1]

create_submission(probs,path,'lgb1.csv')

lg

### for stacking

x_train, y_train = train.drop('stroke', axis=1).reset_index(drop= True), train.stroke

x_test = test

clf = lg

trn = x_train.copy()

tst = x_test.copy()

x_train['lgb_soft'] = 0

kf = KFold(n_splits=5, shuffle=True, random_state=1)

for train_index, val_index in StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        x_train, y_train):
    x_trn, y_trn = trn.loc[train_index,:], y_train[train_index]
    x_val, y_val = trn.loc[val_index,:], y_train[val_index]

    clf.fit(x_trn, y_trn)
    
    probs = clf.predict_proba(x_val)[:, 1]
    
    
    x_train.loc[val_index,'lgb_soft'] = probs

    
clf.fit(trn,y_train)
probs = clf.predict_proba(tst)[:, 1]

x_test.loc[:,'lgb_soft'] = probs

x_train.to_feather('data/mckinsey/stack_trn_lgb')

x_test.to_feather('data/mckinsey/stack_tst_lgb')

# Catboost

In [ ]:
def create_submission(pred, path, fname):
    submission = pd.DataFrame({'id': test.id, 'stroke': pred})
    submission.to_csv(path + fname, index=False)

path = 'data/mckinsey/'

train = pd.read_csv(path + 'train.csv')

test = pd.read_csv(path + 'test.csv')

sample = pd.read_csv(path + 'sample_submission_1.csv')

train.head(1)

test.head(1)

train.shape

train.isnull().sum()

train.smoking_status.unique()

train.work_type.unique()

train.Residence_type.unique()

train.groupby('stroke').size()

def train_cats(df):
    """Change any columns of strings in a panda's dataframe to a column of
    catagorical values. This applies the changes inplace.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values.

    Examples:
    ---------

    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category
    """
    for n, c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

train_cats(train)

def apply_cats(df, trn):
    """Changes any columns of strings in df into categorical variables using trn as
    a template for the category codes.

    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values. The category codes are determined by trn.

    trn: A pandas dataframe. When creating a category for df, it looks up the
        what the category's code were in trn and makes those the category codes
        for df.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category {a : 1, b : 2}

    >>> df2 = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['b', 'a', 'a']})
    >>> apply_cats(df2, df)

           col1 col2
        0     1    b
        1     2    a
        2     3    a

    now the type of col is category {a : 1, b : 2}
    """
    for n, c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name == 'category'):
            df[n] = pd.Categorical(
                c, categories=trn[n].cat.categories, ordered=True)

apply_cats(test, train)

train['bmi_is_na'] = 0

train.loc[train.bmi.isna(),'bmi_is_na'] = 1

test['bmi_is_na'] = 0

test.loc[test.bmi.isna(),'bmi_is_na'] = 1

train.bmi.fillna(train.bmi[train.bmi.notnull()].median(), inplace=True)

test.bmi.fillna(train.bmi[train.bmi.notnull()].median(), inplace=True)

train.dtypes

test.dtypes

cat_cols = [
    'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'
]

for c in cat_cols:
    train[c] = train[c].cat.codes
    test[c] = test[c].cat.codes

#### Catboost

train.dtypes

x_train,y_train = train.drop('stroke',axis=1), train.stroke

cat_cols_index = np.where(x_train.dtypes != "float")[0]

cat_cols_index

from sortedcontainers import SortedList
import copy
import collections
import numpy as np
from itertools import product, chain
import pandas
from sklearn.model_selection import KFold
import catboost as cb
''' a class for doing grid search on a set of parameters provided in a dict. 'pdict' should be a dictionary like the following:
pdict = {'depth':[1,2], 'iterations':[250,100,500], 'thread_count':4}

when grid_search is called it will return an iterator that provides samples from the dictionary e.g.
{'depth':1, 'iterations':250, 'thread_count':4}
{'depth':2, 'iterations':250, 'thread_count':4}
{'depth':1, 'iterations':100, 'thread_count':4}
etc.
after calling an iteration of grid_search, you need to test the classifier and run 'register_result'
This will update the internal list of results, so that the next call to grid_search will use the best
parameters for all the parameters not currently being updated.

grid_search can be provided a list e.g. grid_search(['depth']) this will use the current best parameters for all
the other arguments and only search over 'depth'. You can then call e.g. grid_search(['iterations']) and it will use
the best depth found previously and cycle through all the 'iterations'. Searching incrementally can be much faster
than doing a full grid search, but may miss the global optimum. '''


class paramsearch:
    def __init__(self, pdict):
        self.pdict = {}
        # if something is not passed in as a sequence, make it a sequence with 1 element
        #   don't treat strings as sequences
        for a, b in pdict.items():
            if isinstance(b, collections.Sequence) and not isinstance(b, str):
                self.pdict[a] = b
            else:
                self.pdict[a] = [b]
        # our results are a sorted list, so the best score is always the final element
        self.results = SortedList()

    def grid_search(self, keys=None):
        # do grid search on only the keys listed. If none provided, do all
        if keys == None: keylist = self.pdict.keys()
        else: keylist = keys

        listoflists = []  # this will be list of lists of key,value pairs
        for key in keylist:
            listoflists.append([(key, i) for i in self.pdict[key]])
        for p in product(*listoflists):
            # do any changes to the current best parameter set
            if len(self.results) > 0: template = self.results[-1][1]
            else: template = {a: b[0] for a, b in self.pdict.items()}
            # if our updates are the same as current best, don't bother
            if self.equaldict(dict(p), template): continue
            # take the current best and update just the ones to change
            yield self.overwritedict(dict(p), template)

    def equaldict(self, a, b):
        for key in a.keys():
            if a[key] != b[key]: return False
        return True

    def overwritedict(self, new, old):
        old = copy.deepcopy(old)
        for key in new.keys():
            old[key] = new[key]
        return old

    # save a (score,params) pair to results. Since 'results' is a sorted list,
    #   the best score is always the final element. A small amount of noise is added
    #   because sorted lists don't like it when two scores are exactly the same
    def register_result(self, result, params):
        self.results.add((result + np.random.randn() * 1e-10, params))

    def bestscore(self):
        return self.results[-1][0]

    def bestparam(self):
        return self.results[-1][1]


params = {
    'depth': [1, 3, 5],
    'l2_leaf_reg': [1, 4, 9],
    'iterations': [100],
    'learning_rate': [0.05, 0.08, 0.09, 0.1],
    "one_hot_max_size": [10]
}


# this function does 3-fold crossvalidation with catboostclassifier
def crossvaltest(params, train_set, train_label, cat_dims, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True)
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index, :]
        test = train_set.iloc[test_index, :]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test) == np.ravel(test_labels)))
    return np.mean(res)


def catboost_param_tune(params,
                        train_set,
                        train_label,
                        cat_dims=None,
                        n_splits=5):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually
    #   but 'iterations','learning_rate' together
    for prms in chain(
            ps.grid_search(['iterations', 'learning_rate']),
            ps.grid_search(['one_hot_max_size'])):
        res = crossvaltest(prms, train_set, train_label, cat_dims, n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res, prms)
        print(res, prms, 'best:', ps.bestscore(), ps.bestparam())
    return ps.bestparam()


bestparams = catboost_param_tune(
    params, x_train, y_train, cat_dims=cat_cols_index)

bestparams

clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=10, \
                        depth=1, iterations= 100, l2_leaf_reg= 1, learning_rate= 0.09)
clf.fit(x_train,y_train, cat_features=cat_cols_index)

probs = clf.predict_proba(test)[:,1]

np.mean(probs)

np.mean(probs>0.5)

create_submission(probs,path,'cb2.csv')



test.head()

x_train, y_train = train.drop('stroke', axis=1).reset_index(drop= True), train.stroke

x_test = test

clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=10, \
                        depth=1, iterations= 100, l2_leaf_reg= 1, learning_rate= 0.09)

trn = x_train.copy()

tst = x_test.copy()

x_train['cat_soft'] = 0

kf = KFold(n_splits=5, shuffle=True, random_state=1)

for train_index, val_index in StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        x_train, y_train):
    x_trn, y_trn = trn.loc[train_index,:], y_train[train_index]
    x_val, y_val = trn.loc[val_index,:], y_train[val_index]

    clf.fit(x_trn, y_trn,cat_features=cat_cols_index)
    
    probs = clf.predict_proba(x_val)[:, 1]
    
    
    x_train.loc[val_index,'cat_soft'] = probs

    
clf.fit(trn,y_train)
probs = clf.predict_proba(tst)[:, 1]

x_test.loc[:,'cat_soft'] = probs

x_train.to_feather('data/mckinsey/stack_trn_cat')

x_test.to_feather('data/mckinsey/stack_tst_cat')



# Stacking

In [2]:
rf_trn = pd.read_feather('data/mckinsey/stack_trn_rf')

rf_tst = pd.read_feather('data/mckinsey/stack_tst_rf')

In [5]:
xgb_trn = pd.read_feather('data/mckinsey/stack_trn_xgb')

xgb_tst = pd.read_feather('data/mckinsey/stack_tst_xgb')

In [6]:
lgb_trn = pd.read_feather('data/mckinsey/stack_trn_lgb')

lgb_tst = pd.read_feather('data/mckinsey/stack_tst_lgb')

In [7]:
cat_trn = pd.read_feather('data/mckinsey/stack_trn_cat')

cat_tst = pd.read_feather('data/mckinsey/stack_tst_cat')

In [10]:
trn_final = rf_trn.copy()

In [11]:
tst_final = rf_tst.copy()

In [14]:
trn_final = trn_final.assign(xgb_soft = xgb_trn.xgb_soft,cat_soft = cat_trn.cat_soft,lgb_soft = lgb_trn.lgb_soft)

In [15]:
tst_final = tst_final.assign(xgb_soft = xgb_tst.xgb_soft,cat_soft = cat_tst.cat_soft,lgb_soft = lgb_tst.lgb_soft)

In [18]:
trn_final['stroke'] = train.stroke

In [19]:
trn_final.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,bmi_is_na,rf_soft,xgb_soft,cat_soft,lgb_soft,stroke
0,30669,1,3.0,0,0,0,4,0,95.12,18.0,-1,0,0.000064,0.004896,0.002196,0.003997,0
1,30468,1,58.0,1,0,1,2,1,87.96,39.2,1,0,0.031925,0.031090,0.024035,0.028025,0
2,16523,0,8.0,0,0,0,2,1,110.89,17.6,-1,0,0.000847,0.005516,0.001527,0.004317,0
3,56543,0,70.0,0,0,1,2,0,69.04,35.9,0,0,0.026802,0.033358,0.030217,0.032060,0
4,46136,1,14.0,0,0,0,1,0,161.28,19.1,-1,0,0.002722,0.005769,0.002361,0.004060,0


In [22]:
trn_final.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'bmi_is_na', 'rf_soft', 'xgb_soft', 'cat_soft',
       'lgb_soft', 'stroke'],
      dtype='object')

In [23]:
x_train,y_train = trn_final.loc[:,['rf_soft', 'xgb_soft', 'cat_soft','lgb_soft']],trn_final.stroke

In [24]:
x_test = tst_final.loc[:,['rf_soft', 'xgb_soft', 'cat_soft','lgb_soft']]

In [25]:
clf = SGDClassifier() # faster than vanilla logit 

grid = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1], # learning rate
    'n_iter': [100, 250, 500, 1000], # number of epochs
    'loss': ['log'], # logistic regression,
    'penalty': ['l2', 'l1', 'None'],
    'n_jobs': [-1]
}

grid_obj = GridSearchCV(clf, grid, 
                        scoring='roc_auc',
                        verbose=10,
                        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1).split(
        x_train, y_train),
                        iid = False) 

grid_obj = grid_obj.fit(x_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] alpha=0.0001, loss=log, n_iter=100, n_jobs=-1, penalty=l2 .......
[CV]  alpha=0.0001, loss=log, n_iter=100, n_jobs=-1, penalty=l2, score=0.8578249004926417, total=   0.4s
[CV] alpha=0.0001, loss=log, n_iter=100, n_jobs=-1, penalty=l2 .......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  alpha=0.0001, loss=log, n_iter=100, n_jobs=-1, penalty=l2, score=0.8624255144472439, total=   0.4s
[CV] alpha=0.0001, loss=log, n_iter=100, n_jobs=-1, penalty=l2 .......


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


[CV]  alpha=0.0001, loss=log, n_iter=100, n_jobs=-1, penalty=l2, score=0.877546220436655, total=   0.4s
[CV] alpha=0.0001, loss=log, n_iter=100, n_jobs=-1, penalty=l1 .......


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    4.1s remaining:    0.0s


[CV]  alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l2, score=0.8578246307884586, total=   1.0s
[CV] alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l2 .......
[CV]  alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l2, score=0.862425784151427, total=   1.1s
[CV] alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l2 .......
[CV]  alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l2, score=0.8775470296061638, total=   1.0s
[CV] alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l1 .......
[CV]  alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l1, score=0.856932719055086, total=   1.3s
[CV] alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l1 .......
[CV]  alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l1, score=0.862397465212206, total=   1.4s
[CV] alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l1 .......
[CV]  alpha=0.0001, loss=log, n_iter=250, n_jobs=-1, penalty=l1, score=0.877367124252024, total=   1.2s
[CV] alpha=0.0001, 

[CV]  alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=l2, score=0.8775160114416569, total=   2.4s
[CV] alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=l1 ........
[CV]  alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=l1, score=0.8566024662829316, total=   2.5s
[CV] alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=l1 ........
[CV]  alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=l1, score=0.8548956433604493, total=   2.2s
[CV] alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=l1 ........
[CV]  alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=l1, score=0.5, total=   2.1s
[CV] alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=None ......
[CV]  alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=None, score=0.8578391948143438, total=   1.9s
[CV] alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=None ......
[CV]  alpha=0.001, loss=log, n_iter=500, n_jobs=-1, penalty=None, score=0.8623300391664415, total=   1.9s
[CV] alpha=0.001, loss=log, n_it

[CV]  alpha=0.01, loss=log, n_iter=1000, n_jobs=-1, penalty=l1, score=0.5, total=   4.6s
[CV] alpha=0.01, loss=log, n_iter=1000, n_jobs=-1, penalty=None ......
[CV]  alpha=0.01, loss=log, n_iter=1000, n_jobs=-1, penalty=None, score=0.8579203757734442, total=   4.2s
[CV] alpha=0.01, loss=log, n_iter=1000, n_jobs=-1, penalty=None ......
[CV]  alpha=0.01, loss=log, n_iter=1000, n_jobs=-1, penalty=None, score=0.8622636919374091, total=   4.3s
[CV] alpha=0.01, loss=log, n_iter=1000, n_jobs=-1, penalty=None ......
[CV]  alpha=0.01, loss=log, n_iter=1000, n_jobs=-1, penalty=None, score=0.8774723162881777, total=   4.3s
[CV] alpha=0.1, loss=log, n_iter=100, n_jobs=-1, penalty=l2 ..........
[CV]  alpha=0.1, loss=log, n_iter=100, n_jobs=-1, penalty=l2, score=0.857929006307302, total=   0.4s
[CV] alpha=0.1, loss=log, n_iter=100, n_jobs=-1, penalty=l2 ..........
[CV]  alpha=0.1, loss=log, n_iter=100, n_jobs=-1, penalty=l2, score=0.862359976330761, total=   0.4s
[CV] alpha=0.1, loss=log, n_iter=100

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed:  4.8min finished


In [27]:
clf = grid_obj.best_estimator_

In [28]:
clf.fit(x_train,y_train)

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=1000,
       n_jobs=-1, penalty='None', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [30]:
probs = clf.predict_proba(x_test)[:,1]

In [31]:
def create_submission(pred, path, fname):
    submission = pd.DataFrame({'id': test.id, 'stroke': pred})
    submission.to_csv(path + fname, index=False)

In [32]:
path

'data/mckinsey/'

In [33]:
create_submission(probs,path,'stack.csv')